In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
model_name = "play_model_1-best-2-pmx"
training_files = 2
data_files = glob.glob('play_data/play_model_1-20240906*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['play_pos'].astype('category')

In [3]:
train_data.head()

,run_id,key,play_pos,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,next_seen,...,throw_off_void,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg
0,77127b,9♦ J♦ 10♣ Q♣ Q♠,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
1,77127b,9♦ J♦ 10♣ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
2,77127b,10♣ 9♦ J♦ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
3,77127b,10♣ 9♦ J♦ Q♣ Q♠,20,0,0,0,3,7,7,3,...,0,0,0,0,3,3,3.0,2,2,2.0
4,77127b,9♦ J♦ Q♠ 10♣ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0


In [4]:
label = 'points_max'
problem_type = 'regression'
presets = 'best_quality'
train_data[label].describe()

count    381494.000000
mean         -0.085246
std           1.602664
min          -4.000000
25%          -1.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_max, dtype: float64

In [5]:
exclude = ['run_id', 'key']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
assert label in outputs
outputs.remove(label)

predictor = TabularPredictor(problem_type=problem_type, label=label, path=f"models/{model_name}")
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets=presets)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       10.90 GB / 15.34 GB (71.1%)
Disk Space Avail:   115.34 GB / 452.95 GB (25.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to

(_ray_fit pid=35988) [1000]	valid_set's rmse: 0.680358
(_ray_fit pid=35987) [2000]	valid_set's rmse: 0.629006 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=35985) [3000]	valid_set's rmse: 0.584646 [repeated 8x across cluster]


(_dystack pid=34837) 	-0.5742	 = Validation score   (-root_mean_squared_error)
(_dystack pid=34837) 	211.26s	 = Training   runtime
(_dystack pid=34837) 	795.64s	 = Validation runtime
(_dystack pid=34837) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 242.57s of remaining time.
(_dystack pid=34837) 	Ensemble Weights: {'KNeighborsDist_BAG_L1': 0.545, 'LightGBMXT_BAG_L1': 0.455}
(_dystack pid=34837) 	-0.4873	 = Validation score   (-root_mean_squared_error)
(_dystack pid=34837) 	0.5s	 = Training   runtime
(_dystack pid=34837) 	0.0s	 = Validation runtime
(_dystack pid=34837) Fitting 106 L2 models ...
(_dystack pid=34837) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 242.05s of the 242.02s of remaining time.
(_dystack pid=34837) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=4.53%)


(_ray_fit pid=36749) [1000]	valid_set's rmse: 0.400548 [repeated 8x across cluster]
(_ray_fit pid=36748) [2000]	valid_set's rmse: 0.384608 [repeated 8x across cluster]
(_ray_fit pid=36748) [3000]	valid_set's rmse: 0.376534 [repeated 8x across cluster]


(_dystack pid=34837) 	-0.3774	 = Validation score   (-root_mean_squared_error)
(_dystack pid=34837) 	195.99s	 = Training   runtime
(_dystack pid=34837) 	780.8s	 = Validation runtime
(_dystack pid=34837) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -57.23s of remaining time.
(_dystack pid=34837) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 1.0}
(_dystack pid=34837) 	-0.3774	 = Validation score   (-root_mean_squared_error)
(_dystack pid=34837) 	0.42s	 = Training   runtime
(_dystack pid=34837) 	0.01s	 = Validation runtime
(_dystack pid=34837) AutoGluon training complete, total runtime = 953.79s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 26.2 rows/s (42389 batch size)
(_dystack pid=34837) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("models/play_model_1-best-2-pmx/ds_sub_fit/sub_fit_ho")
(_dystack pid=34837) Deleting DyStack predictor artifacts (clean_up_fits=True) ...
Leaderboard on holdout data (DyStack):


In [6]:
test_data = TabularDataset(data_files[5])[:100000]
y_pred = predictor.predict(test_data[:1000])
y_pred.head(10)

Loaded data from: play_data/play_model_1-20240906_173932.tsv | Columns = 55 / 55 | Rows = 194348 -> 194348


0   -1.333549
1   -1.381132
2   -1.430816
3   -1.333549
4   -1.430816
5   -1.381132
6   -1.333549
7   -1.381132
8   -1.333549
9   -1.430816
Name: points_max, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -0.8909080088522806,
 'mean_squared_error': -0.7937170802371354,
 'mean_absolute_error': -0.5244889637059771,
 'r2': 0.6839774583518725,
 'pearsonr': 0.8274197067716218,
 'median_absolute_error': -0.17490935325622559}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,-0.770694,-0.466708,root_mean_squared_error,191.883895,1482.496526,298.747783,191.883895,1482.496526,298.747783,1,True,4
1,LightGBMXT_BAG_L1,-0.799684,-0.503497,root_mean_squared_error,311.922720,2456.045614,503.965795,311.922720,2456.045614,503.965795,1,True,3
2,WeightedEnsemble_L2,-0.847545,-0.446213,root_mean_squared_error,257.555295,1732.519799,299.481257,0.002729,0.003894,0.506884,2,True,5
3,LightGBMXT_BAG_L2,-0.890908,-0.321812,root_mean_squared_error,944.426102,7060.504136,1355.049773,317.814466,2618.498999,551.892995,2,True,6
4,WeightedEnsemble_L3,-0.890908,-0.321812,root_mean_squared_error,944.428971,7060.508036,1355.372520,0.002869,0.003901,0.322747,3,True,7
5,KNeighborsUnif_BAG_L1,-1.432287,-0.635544,root_mean_squared_error,57.136351,253.443617,0.216610,57.136351,253.443617,0.216610,1,True,1
6,KNeighborsDist_BAG_L1,-1.435061,-0.572725,root_mean_squared_error,65.668671,250.019379,0.226589,65.668671,250.019379,0.226589,1,True,2
